In [5]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd 
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker 

## Webscraping 

In [3]:
# get html texts from relevant websites and initialize datastructure 
nber_page = requests.get("https://www.nber.org/career-resources/research-assistant-positions-not-nber").text
nber_html = BeautifulSoup(nber_page, "lxml")
predoc_page = requests.get("https://predoc.org/opportunities", verify=False).text
predoc_html = BeautifulSoup(predoc_page, "lxml")
job_listing = []
rem_nl = lambda s: s.replace("\n", ", ")
rem_nl2 = lambda s: s.replace("\n", "")

# get nber predoc fellowships 
main_text = nber_html.find("div", class_ = "page-header__intro-inner")
for p in main_text.find_all("p"): 
    if p.em != None :
        if "predoc" in p.text.lower() or "pre-doc" in p.text.lower():
            job_listing.append([p.em.text,  p.a.get("href"),rem_nl(p.text)])
            
# Get predoc website job_listing 
main_text2 = predoc_html.find_all("p") 
for p in main_text2:
    if p.em != None :
        if "predoc" in p.strong.text.lower() or "pre-doc" in p.strong.text.lower():
            job_listing.append([rem_nl2(p.em.text) ,p.a.get("href"), rem_nl2(p.strong.text)])


C:\Users\user\anaconda3\envs\py3-TF2.0\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


## Data management 

In [29]:
# Make a panda dataframe from the data 
job_listing = pd.DataFrame(job_listing)
job_listing.columns = ["Institution", "website_link", "summary"]

# Turn into an sqltable for easier interface 
def access_database(dbfile, query):
    connect = sqlite3.connect(dbfile)
    cursor = connect.cursor()
    cursor.execute(query)
    connect.commit()
    connect.close()

ourschema= "CREATE TABLE predoctoral_fellowship (id INTEGER PRIMARY KEY AUTOINCREMENT,  institution VARCHAR, url VARCHAR, summary VARCHAR)"

access_database("job_vacancies.db", "DROP TABLE IF EXISTS predoctoral_fellowship")
access_database("job_vacancies.db", ourschema)
engine = create_engine('sqlite:///job_vacancies.db')
db = scoped_session(sessionmaker(bind=engine))

job_listing.to_sql("predoctoral_fellowship", con = engine, if_exists="replace")

# Find vacancy by institution, make the search as sql robust as possible (this search statement uses the LIKE function)

def find_by_institution(uni):
    connect = sqlite3.connect("job_vacancies.db")
    cursor = connect.cursor()
    rows = cursor.execute(f"SELECT * FROM predoctoral_fellowship WHERE Institution LIKE '%{uni}%'").fetchall()
    connect.commit()
    connect.close()
    return pd.DataFrame(rows)

find_by_institution("Chicago")



,0,1,2,3
0,2,University of Chicago (Becker Friedman Institute),https://bfi.uchicago.edu/predoctoral-research-...,"Pre-Doctoral Research Fellows, NBER Sponsoring..."
1,8,University of Chicago,https://www.nber.org/sites/default/files/2021-...,"Pre-Doctoral Research Fellow, NBER Sponsoring ..."
2,12,University of Chicago (Becker Friedman Institute,https://boards.greenhouse.io/universityofchica...,"Pre-Doctoral Research Professional, NBER Spons..."
3,13,University of Chicago,https://uchicago.wd5.myworkdayjobs.com/en-US/E...,Predoctoral Fellow (BA- or MA-level Research P...
4,15,University of Chicago (Becker Friedman Institu...,https://bfi.uchicago.edu/predoctoral-research-...,"Pre-Doctoral Research Professional, NBER Spons..."
5,16,University of Chicago (Becker Friedman Institu...,https://bfi.uchicago.edu/predoctoral-research-...,"Pre-Doctoral Research Professional, NBER Spons..."
6,19,University of Chicago (Becker Friedman Institute),https://bfi.uchicago.edu/predoctoral-research-...,"Pre-Doctoral Research Fellows, NBER Sponsoring..."
7,23,University of Chicago,/sites/default/files/2020-09/2021FMCRPJobPosti...,Full-Time Research Professional (pre-doctoral)...
8,34,"Becker Friedman Institute, The University of C...",https://boards.greenhouse.io/universityofchica...,Pre-Doctoral Research Professional
9,41,"The University of Chicago, Harris School of Pu...",https://www.nber.org/sites/default/files/2021-...,Pre-Doctoral Research Fellow
